# ETL the data in the camera reviews dataset from AWS S3 datasets in order to match the schema.

In [ ]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark
 
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"
 
# Start a SparkSession
import findspark
findspark.init()

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-09-22 01:21:08--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2020-09-22 01:21:08 (8.88 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AmazonHWCamera").config("spark.driver.extraClassPath2","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# load csv from AWS S3
from pyspark import SparkFiles
# Load in user_data.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

In [ ]:
# make header 'true' and sep '\t' and verify data is connected
df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Camera_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    2975964|R1NBG94582SJE2|B00I01JQJM|     860486164|GoPro Rechargeabl...|          Camera|          5|            0|          0|   N|                Y|          Five Stars|                  ok|2015-08-31 00:00:00|
|         US|   23526356|R273DCA6Y0H9V7|B00TCO0ZAA|     292641483|Professional 58mm...| 

In [ ]:
# print df schema
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



# Count number of records(rows) in this dataset

In [ ]:
# count number of records(rows) in this dataset
row_number = df.count()
print(f"Number of records in this dataset: {row_number}")

Number of records in this dataset: 1801974


# Create user dataframe to match **review_id_table** table in the schema

In [ ]:
# make df for review
review_df = df.select(["review_id", "customer_id", "product_id", "product_parent","review_date"])
review_df.show(10)

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R1NBG94582SJE2|    2975964|B00I01JQJM|     860486164|2015-08-31 00:00:00|
|R273DCA6Y0H9V7|   23526356|B00TCO0ZAA|     292641483|2015-08-31 00:00:00|
| RQVOXO7WUOFK6|   52764145|B00B7733E0|      75825744|2015-08-31 00:00:00|
|R1KWKSF21PO6HO|   47348933|B006ZN4U34|     789352955|2015-08-31 00:00:00|
|R38H3UO1J190GI|   33680700|B00HUEBGMU|      19067902|2015-08-31 00:00:00|
|R3NPIFKLR19NQA|   30301059|B008MW6Y12|     597683407|2015-08-31 00:00:00|
|R3MBE6UCH3435E|   28282645|B00TE8XKIS|      35563334|2015-08-31 00:00:00|
|R2E7A4FF0PVY5Q|     502818|B00ZKDUFBQ|     555190742|2015-08-31 00:00:00|
| R3R8JDQ2BF4NM|    1481233|B010BZ7S2Q|     129544315|2015-08-31 00:00:00|
|R1YND4BS823GN5|   27885926|B00HRXSSRA|     708418657|2015-08-31 00:00:00|
+--------------+---------

In [ ]:
# double checking that datatype matches schema
review_df.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'timestamp')]

In [ ]:
# change review_date from timestamp to yyyy-mm-dd format and verify change
from pyspark.sql.types import DateType
review_df = review_df.withColumn("review_date", review_df['review_date'].cast(DateType()))
review_df.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

# Create user dataframe to match **products** table in the schema

In [ ]:
# make df for products and drop duplicates
product_df = df.select(["product_id", "product_title"]).drop_duplicates()
product_df.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00L9ZF5BY|Flylinktech Actio...|
|B00017LSF8|B+W Clear UV Haze...|
|B00CGTN2N0|Queen Spy Mini U8...|
|B002POMZWU|Usb cable for kod...|
|B00FEY2BCW|Camera Case Eveca...|
|B0131XNIDS|*5-pack* microSue...|
|B004OUMTT2|Celestron PowerSe...|
|B004HY3XOA|58MM UV Lens Filt...|
|B00LBFKLRU|(Create a generic...|
|B007QXZ6D0|Vello Cinch Strap...|
+----------+--------------------+
only showing top 10 rows



In [ ]:
# double checking that datatype matches schema
product_df.dtypes

[('product_id', 'string'), ('product_title', 'string')]

**bold text**# Create user dataframe to match **customers** table in the schema

In [ ]:
# count the number of customers
customer = df.groupby('customer_id').agg({"customer_id": "count"})
#rename count(customer_id) column to match schema
customer_df = customer.withColumnRenamed('count(customer_id)', 'customer_count')
customer_df.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   52695798|             1|
|   48363612|             5|
|   46909180|             1|
|   45595220|             1|
|   50372387|             2|
|    9731896|             1|
|   24540309|             1|
|    2019000|             1|
|   50798385|             5|
|   37669883|             1|
+-----------+--------------+
only showing top 10 rows



In [ ]:
# double checking that datatype matches schema
customer_df.dtypes

[('customer_id', 'int'), ('customer_count', 'bigint')]

In [ ]:
# change customer_count from bigint to int & verify change
from pyspark.sql.types import IntegerType
customer_df = customer_df.withColumn("customer_count",customer_df["customer_count"].cast(IntegerType()))
customer_df.dtypes

[('customer_id', 'int'), ('customer_count', 'int')]

# Create user dataframe to match **vine_table** table in the schema

In [ ]:
# make df to match schema
vine_df = df.select(["review_id", "star_rating","helpful_votes","total_votes","vine"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1NBG94582SJE2|          5|            0|          0|   N|
|R273DCA6Y0H9V7|          5|            0|          0|   N|
| RQVOXO7WUOFK6|          2|            1|          1|   N|
|R1KWKSF21PO6HO|          5|            0|          0|   N|
|R38H3UO1J190GI|          5|            1|          1|   N|
|R3NPIFKLR19NQA|          3|            0|          0|   N|
|R3MBE6UCH3435E|          3|            8|          8|   N|
|R2E7A4FF0PVY5Q|          5|            0|          1|   N|
| R3R8JDQ2BF4NM|          5|            0|          2|   N|
|R1YND4BS823GN5|          1|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [ ]:
# double checking that datatype matches schema
vine_df.dtypes

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string')]

# Load the DataFrames that correspond to tables into an RDS instance


In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<insert endpoint>:5432/postgres"
config = {"user":"<insert user>", 
          "password": "<insert password>", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to review_df table in RDS
review_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write DataFrame to product_df table in RDS
product_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write DataFrame to customer_df table in RDS
customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write DataFrame to vine_df table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

# Are Vine Reviews Trustworthy?

## A Comparison between Vine and Non-Vine Reviewers

Count vine members

In [ ]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Find vine users
vine_users = vine_df.filter(col("vine")  == "Y")
vine_users.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2H4QESQ10P8EN|          5|            4|          5|   Y|
|  RY5OEUWERD2Z|          4|            5|          5|   Y|
|R3GNSMDV7WZGKP|          4|            1|          1|   Y|
|R1RI2CNUVSAOU6|          4|            0|          0|   Y|
|R3UX0VTCK4SUJX|          4|            0|          0|   Y|
|R2XDFESUGNQYDC|          5|            2|          3|   Y|
|R1VYRVEFO3I4NX|          4|           11|         12|   Y|
| RX7LRJLKAX9SY|          5|            1|          3|   Y|
|R2Z3DTPH8Y206X|          5|            0|          0|   Y|
|R29DFG91V74EY5|          1|            2|          3|   Y|
|R37L6YZDX0P3J9|          1|           28|         35|   Y|
|R3TY6J21QQGUUB|          4|            0|          0|   Y|
|R2JT4XRQO2KASL|          5|           38|         41|   Y|
|R1SCZZ7KYLMTMO|          4|            

In [ ]:
# count rows to verify
viners = vine_users.count()
print(f"Number of vine users in this dataset: {viners}")

Number of vine users in this dataset: 7883


Total vote count by star rating amongst vine users.

In [ ]:
from pyspark.sql.functions import desc
vine_reviews_df = vine_users.select(["star_rating","total_votes"])\
         .groupBy("star_rating")\
         .agg({"total_votes": "count"})

vine_reviews_df2 = vine_reviews_df.withColumnRenamed('count(total_votes)','count_of_total_votes')
vine_reviews_df2.orderBy(desc("count_of_total_votes")).show(truncate=False)

+-----------+--------------------+
|star_rating|count_of_total_votes|
+-----------+--------------------+
|5          |3293                |
|4          |2951                |
|3          |1139                |
|2          |357                 |
|1          |143                 |
+-----------+--------------------+



Get total votes and helpful votes for vine users by star rating.

In [ ]:
vine_tots_reviews = vine_users.select(["star_rating","total_votes", "helpful_votes"])\
         .groupBy("star_rating")\
         .agg({"total_votes": "sum", "helpful_votes": "sum"})

vine_df1 = vine_tots_reviews.withColumnRenamed('sum(total_votes)','sum_of_total_votes')\
                            .withColumnRenamed("sum(helpful_votes)", "sum_of_helpful_votes")
vine_df2 = vine_df1.orderBy(desc("sum_of_total_votes"))
vine_df2.show(truncate=False)

+-----------+------------------+--------------------+
|star_rating|sum_of_total_votes|sum_of_helpful_votes|
+-----------+------------------+--------------------+
|5          |34292             |30045               |
|4          |26484             |23104               |
|3          |12791             |10682               |
|2          |3089              |2269                |
|1          |1500              |1094                |
+-----------+------------------+--------------------+



Count non-vine members

In [ ]:
# Find non-vine users
novine_users = vine_df.filter(col("vine")  == "N")
novine_users.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1NBG94582SJE2|          5|            0|          0|   N|
|R273DCA6Y0H9V7|          5|            0|          0|   N|
| RQVOXO7WUOFK6|          2|            1|          1|   N|
|R1KWKSF21PO6HO|          5|            0|          0|   N|
|R38H3UO1J190GI|          5|            1|          1|   N|
|R3NPIFKLR19NQA|          3|            0|          0|   N|
|R3MBE6UCH3435E|          3|            8|          8|   N|
|R2E7A4FF0PVY5Q|          5|            0|          1|   N|
| R3R8JDQ2BF4NM|          5|            0|          2|   N|
|R1YND4BS823GN5|          1|            0|          0|   N|
|R2TZNSA18V7YF6|          4|            1|          1|   N|
|R22ZVRDPPXIDNL|          5|            0|          0|   N|
|R1F4O6WOO2W461|          3|            3|          3|   N|
| RT1KLS3QSJNUT|          5|            

In [ ]:
# count rows to verify
no_viners = novine_users.count()
print(f"Number of non-vine users in this dataset: {no_viners}")

Number of non-vine users in this dataset: 1794089


In [ ]:
novine_reviews_df = novine_users.select(["star_rating","total_votes"])\
         .groupBy("star_rating")\
         .agg({"total_votes": "count"})

novine_reviews_df2 = novine_reviews_df.withColumnRenamed('count(total_votes)','count_of_total_votes')
novine_reviews_df2.orderBy(desc("count_of_total_votes")).show(truncate=False)

+-----------+--------------------+
|star_rating|count_of_total_votes|
+-----------+--------------------+
|5          |1059413             |
|4          |333749              |
|1          |170014              |
|3          |140321              |
|2          |90592               |
+-----------+--------------------+



In [ ]:
novine_tots_reviews = novine_users.select(["star_rating","total_votes", "helpful_votes"])\
         .groupBy("star_rating")\
         .agg({"total_votes": "sum", "helpful_votes": "sum"})

nonvine_df1 = novine_tots_reviews.withColumnRenamed('sum(total_votes)','sum_of_total_votes')\
                            .withColumnRenamed("sum(helpful_votes)", "sum_of_helpful_votes")
df2 = nonvine_df1.orderBy(desc("sum(total_votes)"))
df2.show(truncate=False)

+-----------+------------------+--------------------+
|star_rating|sum_of_total_votes|sum_of_helpful_votes|
+-----------+------------------+--------------------+
|5          |3174611           |2762260             |
|4          |1208102           |1038050             |
|1          |1003436           |597930              |
|3          |649617            |478698              |
|2          |442456            |291343              |
+-----------+------------------+--------------------+

